In [1]:
!pip install scanpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.1/122.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.6 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=9c1fcb881e03497083d9003f9296e778c34dfd8edd45594c6d76f04c62c8b3c6
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8026 sha256=f7d6670330de4ffb83f0259feab60cc2149dd7449ebc4a1e8c45174903228c6d
  Stored in directory: /root/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9bd9

In [2]:
import numpy as np
import pandas as pd
import sys
import scanpy as sc
import os,gc
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%run '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/C2CGNN/Preprocessing/image processing/match.ipynb'
%run '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/C2CGNN/Preprocessing/image processing/GMD.py'
%run '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/C2CGNN/Preprocessing/image processing/ct_dataproc.ipynb'

In [5]:
#sub functions
def ext_ref(d_data,col):
  ### extract and reformat the selected col in df as cell*time. the cells are filtered by cellindex before reformating.
  #filter the df with cellind
  ind_c = list(d['cellindex'])
  df_c = df[df['nu_track_id'].isin(ind_c)]
  #reformat to cellXtime
  d_arr = np.zeros(d_data.shape)
  for i in range(len(ind_c)):
    df_i = df_c[df_c['nu_track_id'] == d['cellindex'][i]]
    #get values
    values = df_i[col].values
    #get frame ranges
    fr = df_i['frame'].values
    for f,v in zip(fr,values):
      d_arr[i,f] = v
  return d_arr

# Import data

In [6]:
#ERKTR

#Params
#FRET channels
fc = []
#State sampling interval
tsamp = 6
#Declare (new) output channel names
cn = ['ERKTR']
#Declare matching input channel indices (or descriptors)
ci = [[6,2]]
#Set minimum track length
minL = 25
#Set maximum gap size
maxGap = 2
#Set 'start only' flag
sto = False
#Check if 'fc' exists and is not empty
if 'fc' in locals() and fc:
    GMD = mdi
    # Calculate the power ratio for FRET signals
    prat = iman_powerratio_est(GMD, None, fc)
    prat = list(prat.astype(float))

In [8]:
# get data
d_ktr = np.empty((0,228))
d_x = np.empty((0,228))
d_y = np.empty((0,228))
d_well = []
d_t = []
xy = 2
for i in range(xy):
  nu_dir = (f"/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/Data/nu_xy{i+1}.csv")
  cyto_dir = (f"/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/Data/cyto_xy{i+1}.csv")
  df,d,p = ct_dataproc(nu_dir = nu_dir,
    cyto_dir = cyto_dir,
    tsamp = tsamp,
    name = cn,
    ind = ci,
    dlengthmin = minL,
    gapmax = maxGap,
    startonly = sto)
  t_ktr = d['data'][0]
  t_x = ext_ref(t_ktr,'cyto_X')
  t_y = ext_ref(t_ktr,'cyto_Y')
  t_well = [i] * t_ktr.shape[0]
  t_t = ['Control'] * t_ktr.shape[0]
  d_ktr =np.append(d_ktr,t_ktr,axis = 0)
  d_x = np.append(d_x,t_x,axis = 0)
  d_y = np.append(d_y,t_y,axis = 0)
  d_well.extend(t_well)
  d_t.extend(t_t)
#metadata & params
meta = {'treatment':'Control','Cell_type':'MCF10A','reporter':'ERKTR'}

KTR 2
KTR 2


# Export as anndata

In [ ]:
#build a temp anndata for test (only include one well)
filedir = '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/Data'
h5ad_dir = os.path.join(filedir,"xy12.h5ad")

In [ ]:
d_ktr[d_ktr<0] = np.nan
ann = sc.AnnData(d_ktr)

In [ ]:
ann.var['timepoints'] = np.arange(d_ktr.shape[-1])
ann.obsm['x'] = d_x
ann.obsm['y'] = d_y
ann.obs['Well of Origin'] = d_well
ann.obs['Well of Origin'] = ann.obs['Well of Origin'].astype('category')
ann.uns = meta

In [ ]:
ann.write(h5ad_dir)